In [44]:
import pandas as pd
from sqlalchemy import create_engine
from string import digits
import numpy as np
import os
import nltk
import string
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI


In [45]:
csv1 = "./indeed_job_dataset_jds_v2.csv"
# csv2 = "./indeed_job_dataset_V5_CSV.csv"
df1 = pd.read_csv(csv1, encoding = 'unicode_escape')
# df2 = pd.read_csv(csv2)
df1.head()
# df2.head()

,Index_No,Job_Title,Link,Queried_Salary,Job_Type,Skill,duplicate,Description,Unnamed: 8,Unnamed: 9,...,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']","[POSITION SUMMARY, The Business Analyst role i...","[position summary, the business analyst role i...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']","[What do we need?, \r\r\nYou to have an amazin...","[what do we need?, \r\r\nyou to have an amazin...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...","[Validate, analyze, and conduct statistical an...","[validate, analyze, and conduct statistical an...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],"[Full time, Washington, DC metro area, Startin...","[full time, washington, dc metro area, startin...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",[Assist in consultations with business partner...,[assist in consultations with business partner...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [46]:
df = df1[['Job_Type', 'Description']]
df.head()

,Job_Type,Description
0,data_scientist,"[position summary, the business analyst role i..."
1,data_scientist,"[what do we need?, \r\r\nyou to have an amazin..."
2,data_scientist,"[validate, analyze, and conduct statistical an..."
3,data_scientist,"[full time, washington, dc metro area, startin..."
4,data_scientist,[assist in consultations with business partner...


In [47]:

df.dropna(inplace=True)

len(df['Job_Type'])

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


5413

In [48]:
df.head()

,Job_Type,Description
0,data_scientist,"[position summary, the business analyst role i..."
1,data_scientist,"[what do we need?, \r\r\nyou to have an amazin..."
2,data_scientist,"[validate, analyze, and conduct statistical an..."
3,data_scientist,"[full time, washington, dc metro area, startin..."
4,data_scientist,[assist in consultations with business partner...


In [49]:
# df.drop_duplicates(inplace = True)

In [50]:
df.shape


(5413, 2)

In [51]:
#download stop words 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williammdavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [72]:
#show tokenization 
df['Description'].head().apply(process_text)

0    [position, summary, business, analyst, role, p...
1    [need, amazing, personality, communication, st...
2    [validate, analyze, conduct, statistical, anal...
3    [full, time, washington, dc, metro, area, star...
4    [assist, consultations, business, partners, in...
Name: Description, dtype: object

In [73]:
# #example 
# message4 = 'hello world hello hello world play'
# message5= 'test test test one hello'
# print(message4)
# print()

# #convert the text to a matrix of a token counts 
# from sklearn.feature_extraction.text import CountVectorizer
# bow4 =CountVectorizer(analyzer=process_text).fit_transform([[message4],[message5]])
# print(bow4)
# print(bow4.shape)

In [81]:
#convert collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
count_v  = CountVectorizer(analyzer=process_text)
message_bow = count_v.fit_transform(df['Description'])

In [ ]:
# print(message_bow)

In [82]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df['Job_Type'],test_size=0.20, random_state=0)

In [99]:
message_bow.shape

(5413, 34069)

In [91]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier =  MultinomialNB().fit(X_train, y_train)

In [92]:
#print the prediction
print(classifier.predict(X_train))
#print values
print(y_train.values)

['data_analyst' 'data_engineer' 'data_scientist' ... 'data_scientist'
 'data_analyst' 'data_analyst']
['data_analyst' 'data_engineer' 'data_scientist' ... 'data_scientist'
 'data_analyst' 'data_analyst']


In [93]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_train)
print(classification_report(y_train, prediction))
print()

                precision    recall  f1-score   support

  data_analyst       0.89      0.88      0.88      1335
 data_engineer       0.93      0.90      0.92      1067
data_scientist       0.91      0.93      0.92      1928

      accuracy                           0.91      4330
     macro avg       0.91      0.91      0.91      4330
  weighted avg       0.91      0.91      0.91      4330




In [94]:
print('Confusion Matrix: \n', confusion_matrix(y_train,prediction))
print('Accuracy Matrix:',  accuracy_score(y_train,prediction))

Confusion Matrix: 
 [[1178   32  125]
 [  60  962   45]
 [  93   36 1799]]
Accuracy Matrix: 0.9096997690531178


In [95]:
#print the prediction
print(classifier.predict(X_test))
#print values
print(y_test.values)


['data_scientist' 'data_scientist' 'data_scientist' ... 'data_engineer'
 'data_engineer' 'data_scientist']
['data_scientist' 'data_scientist' 'data_scientist' ... 'data_engineer'
 'data_engineer' 'data_scientist']


In [101]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_test)
print(classification_report(y_test, prediction))
print()
print('Confusion Matrix: \n', confusion_matrix(y_test,prediction))
print()
print('Accuracy Matrix:',  accuracy_score(y_test,prediction))

                precision    recall  f1-score   support

  data_analyst       0.83      0.84      0.84       339
 data_engineer       0.89      0.85      0.87       261
data_scientist       0.88      0.88      0.88       483

      accuracy                           0.86      1083
     macro avg       0.86      0.86      0.86      1083
  weighted avg       0.86      0.86      0.86      1083


Confusion Matrix: 
 [[286  14  39]
 [ 16 223  22]
 [ 42  14 427]]

Accuracy Matrix: 0.8642659279778393


In [100]:
example = ['data analyst is cool. i know sql, python, and tableau. I am studious and adaptable.']
input_bow = count_v.transform(example)
print(input_bow)
# classifier =  MultinomialNB().fit(X1_train, y1_train)

classifier.predict(input_bow)

  (0, 1514)	1
  (0, 2264)	1
  (0, 7029)	1
  (0, 7823)	1
  (0, 16762)	1
  (0, 24364)	1
  (0, 28706)	1
  (0, 29902)	1


array(['data_analyst'], dtype='<U14')

In [121]:
example1 = ['Weldon Schmidt \
59222 Yundt Falls, Philadelphia, PA \
+1 (555) 408 5416 Work Experience \
07/2017 - PRESENT \
Lead Big Data Engineer \
Boston, MA \
Support, maintain, and document Hadoop and MySQL data warehouse \
Iterate and improve existing features in the pipeline as well as add new ones \
Design, develop, document, and test new requirements in the data pipeline using PERL, BASH, PIG and OOZIE in the Hadoop ecosystem \
Provide full operational support – analyze code to identify root causes of production issues and provide solutions or workarounds and lead it to resolution \
Participate in full development life cycle including requirements analysis, design, development, deployment and operations support \
Work with engineering team members to explore and create interesting solutions while sharing knowledge within the team \
Work across product teams to help solve customer-facing issues \
Demonstrable experience designing technological solutions to complex data problems, developing & testing modular, reusable, efficient and scalable code to implement those solutions \
10/2011 - 03/2017 \
Dallas, TX \
Plan, develop, monitor and evolve needed infrastructure in collaboration with Ops partners \
Progressive experience as a Systems/Software Engineer, Application Developer or related occupation \
Provides technical design based on business requirements \
Communicates and coordinates with cross functional teams to ensure business objectives are met \
Develops, tests and deploys software applications/systems using scientific analysis and mathematical models to predict and measure outcome and consequences of design \
Provides guidance and mentoring in technical areas to a team of Developer/Analysts \
Documents and submits status reports to leadership \
Provides guidance on new technologies/methodologies \
09/2004 - 09/2011 \
Detroit, MI \
Designing, developing and maintaining new generation machine learning based big-data web page categorization, data/IP mining, and malicious site detection systems \
Creating and enhancing tools to analyze and process large quantity of data set \
Utilize your programming skills for efficient and robust implementation \
Work closely with malware research/data science teams to enhance malicious site detection, and machine learning/data mining based big data system \
Solve engineering problems by creating solutions that leverage existing technologies and utilize your technical prowess to design and implement solutions where none exist \
Design, build, improve and maintain a high performance and highly scalable services exposed to our solution and a broader development community \
Work with real-time data processing, messaging, streaming techniques and workflows \
University of Sioux Falls \
1998 - 2003 \
Bachelors Degree in Computer Science \
Professional Skills \
Very strong skills working with very large data sets with a variety of tools; experience in dealing with performance and scaling issues \
Demonstrated excellent planning and organizational skills \
Proven design, coding, testing and debugging skills and experience \
Strong analytical, learning and problem solving skills with personal interest in subjects such as math/statistics, machine learning, AI and analytics \
Self-starting, requiring minimal supervision with strong problem-solving skills \
Relevant software development / consulting experience with strong skill on database modeling and architecture \
Strong execution skills in a fast-paced environment using agile methodologies ']
input_bow2 = count_v.transform(example1) 
print(input_bow2)
# classifier =  MultinomialNB().fit(X1_train, y1_train)

classifier.predict(input_bow2)
print(what)

  (0, 31)	1
  (0, 388)	1
  (0, 434)	1
  (0, 780)	1
  (0, 1428)	1
  (0, 1523)	1
  (0, 1814)	1
  (0, 2194)	2
  (0, 2308)	1
  (0, 2342)	1
  (0, 2410)	2
  (0, 2701)	1
  (0, 2826)	1
  (0, 2872)	1
  (0, 3679)	3
  (0, 3991)	1
  (0, 3993)	1
  (0, 4463)	1
  (0, 4557)	1
  (0, 4621)	2
  (0, 5118)	1
  (0, 5142)	1
  (0, 5862)	1
  (0, 6001)	2
  (0, 6031)	1
  :	:
  (0, 29416)	1
  (0, 29579)	1
  (0, 29603)	2
  (0, 29793)	1
  (0, 29819)	1
  (0, 30114)	3
  (0, 30167)	3
  (0, 30239)	3
  (0, 30269)	1
  (0, 30311)	1
  (0, 30313)	1
  (0, 30544)	1
  (0, 30561)	2
  (0, 30586)	1
  (0, 30975)	2
  (0, 32066)	3
  (0, 32117)	1
  (0, 32228)	1
  (0, 32694)	1
  (0, 32815)	1
  (0, 32899)	1
  (0, 33125)	1
  (0, 33179)	1
  (0, 33201)	1
  (0, 33210)	1
['data_scientist']
